In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
path = "/content/gdrive/MyDrive/Stage/hittingsetcomplex.json"
with open(path, 'r') as json_file:
    data = json.load(json_file)

my_dict = {int(key): {frozenset(inner_list) for inner_list in value} for key, value in data.items()}

print(my_dict)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
key_count = len(my_dict)
print("The quantity of connected components:", key_count)

The quantity of connected components: 1802


In [ ]:

value_counts = {key: len(value) for key, value in my_dict.items()}

sorted_items = sorted(value_counts.items(), key=lambda x: x[1], reverse=True)
top_10 = sorted_items[:10]

print("top 10 connected component:")
for key, count in top_10:
    print(f"key: {key}, quantity of components: {count}")


top 10 connected component:
key: 423, quantity of components: 1038
key: 204, quantity of components: 1033
key: 302, quantity of components: 944
key: 367, quantity of components: 934
key: 139, quantity of components: 930
key: 360, quantity of components: 913
key: 257, quantity of components: 893
key: 602, quantity of components: 886
key: 294, quantity of components: 868
key: 659, quantity of components: 850


In [ ]:
result_dict = {}

for index, elements in my_dict.items():
    result_dict[index] = set().union(*elements)


In [ ]:
allrepairs = {}

In [ ]:
def findrepairs(mhs, dataset):
    result_dict = {}

    for index, elements1 in mhs.items():
        if index in dataset:
            result_dict[index] = elements1 - dataset[index]
        else:
            result_dict[index] = elements1

    return result_dict


In [ ]:
allrepairs = findrepairs(my_dict,result_dict)

In [ ]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 82.9 MB/s eta 0:00:00


In [ ]:
import pulp

def find_minimal_covering_sets(original_sets):

    lp_problem = pulp.LpProblem("MinimalCoveringSets", pulp.LpMinimize)


    set_vars = [pulp.LpVariable(f"Set_{i}", cat=pulp.LpBinary) for i in range(len(original_sets))]

    lp_problem += pulp.lpSum(set_vars)


    for element in set.union(*original_sets):
        lp_problem += pulp.lpSum(set_vars[i] for i, original_set in enumerate(original_sets) if element in original_set) >= 1


    lp_problem.solve()

    minimal_covering_sets = [original_sets[i] for i, var in enumerate(set_vars) if var.varValue == 1]

    return minimal_covering_sets



In [ ]:
original_sets = [set(fs) for fs in allrepairs[247]]
minimal_covering_sets = find_minimal_covering_sets(original_sets)
print(minimal_covering_sets)


[{424450, 379910, 152840, 289610, 252010, 158190, 194030, 325490, 416600, 328980, 257940, 340850, 407030, 168600, 260670}, {148450, 402340, 96550, 216360, 359050, 189930, 242060, 128210, 277560, 427030, 106040, 462170, 404510}, {184000, 219840, 145220, 451460, 387660, 287120, 52120, 303770, 298010, 282270, 142240, 317730, 397090, 279150, 62710, 340090}, {101640}]


In [ ]:
pip install cplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 11.9 MB/s eta 0:00:00


In [ ]:
'''
original_sets = [set(fs) for fs in allrepairs[302]]
solutions = find_all_solutions(original_sets)

for i, solution in enumerate(solutions):
    print(f'Solution {i + 1}: {solution}')
'''

In [ ]:
import cplex

def find_all_solutions(original_sets):
    sets = original_sets
    num_sets = len(sets)


    problem = cplex.Cplex()

    try:

        problem.variables.add(names=[f'x_{i}' for i in range(num_sets)], types=['B'] * num_sets)

        elements = set.union(*sets)
        for element in elements:
            set_indices = [i for i, s in enumerate(sets) if element in s]
            problem.linear_constraints.add(
                lin_expr=[cplex.SparsePair(ind=[f'x_{i}' for i in set_indices], val=[1] * len(set_indices))],
                senses=['G'],
                rhs=[1]
            )

        problem.objective.set_sense(problem.objective.sense.minimize)
        problem.objective.set_linear([(f'x_{i}', 1) for i in range(num_sets)])


        problem.parameters.mip.pool.intensity = 4
        problem.parameters.mip.pool.absgap = 0.0

        problem.solve()

        num_solutions = problem.solution.pool.get_num()
        all_solutions = set()
        for solution_num in range(num_solutions):
            solution = problem.solution.pool.get_values(solution_num)
            all_solutions.add(frozenset(i for i, value in enumerate(solution) if value > 0.5))

    except cplex.exceptions.CplexError as e:
        print("CPLEX Error:", e)
        all_solutions = set()

    return all_solutions


In [ ]:
output_dict = {}

for key, original_sets in allrepairs.items():
  original_set = [set(fs) for fs in original_sets]
  if len(original_set)<= 1000:
    solutions = find_all_solutions(original_set)
    min =100000
    new_set={}
    for i in solutions:
      if len(i)<min:
        min = len(i)
        new_set = {list(original_sets)[index] for index in i}
      if len(i)==min:
        new_set.update({list(original_sets)[index] for index in i})
    output_dict[key] = [set(fs) for fs in new_set]
  else:
    output_dict[key] =find_minimal_covering_sets(original_set)

Using size restricted mode (Could not find directory for cpxchecklic).
CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Root node processing (before b&c):
  Real time             =    0.00 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.01 ticks)
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 11.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 12 rows and 11 columns.
All rows and columns eliminated.
Presolve time = 0.00 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------

In [ ]:
quantity =0
for key, solution in output_dict.items():
  quantity += len(solution)
    print(f'No. connected component: {key} length of optiaml repairs :{len(solution)}, result : {solution}')


No. connected component: 9 length of optiaml repairs :0, result : []
No. connected component: 31 length of optiaml repairs :21, result : [{70440}, {9940}, {268430}, {194560, 235520, 350210, 63490, 227330, 124940, 8210, 202770, 204820, 309270, 444440, 446490, 30750, 288800, 331810, 378920, 413740, 16430, 231470, 323630, 415790, 182320, 84020, 47160, 139320, 6200, 303160, 200760, 288830, 280640, 415810, 30790, 135240, 436300, 125010, 116820, 331860, 120920, 458840, 409690, 225370, 454750, 426080, 12400, 391280, 247920, 211060, 397430, 73850, 342140, 391300, 403600, 172180, 166040, 110750, 151710, 450720, 92320, 323750, 10410, 307370, 133290, 166060, 137390, 172210, 256180, 159930, 204990, 389310, 280770, 121030, 336070, 127180, 239820, 141520, 51410, 211160, 366810, 49370, 420060, 213210, 73950, 465120, 415970, 397540, 340200, 260330, 198890, 307440, 176370, 329970, 278770, 342260, 159990, 49400, 399610, 82170, 430330, 358650, 168190, 256250, 307460, 389380, 73990, 233740, 448780, 315660